<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-07-07 09:40:06


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv'
df_mypf = read_file(path_mypf)

#mypf = mypf[mypf['Nifty100'] == 1]

def get_stock_list(df_mypf):
  stock_n100 = df_mypf['Symbol'].unique()
  exclude = ['CADILAHC','MMTC']
  stock_n100 = df_mypf[~df_mypf['Symbol'].isin(exclude) ]['Symbol'].unique()
  stock_ids = stock_n100
  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()
  return stock_ids

stock_ids = get_stock_list(df_mypf)

In [5]:
# star stocks & my stocks
star_stocks = list(df_mypf[df_mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(df_mypf[df_mypf['InPortfolio'] == 1]['Symbol'].values)

In [6]:
def print_distribution():
  # conviction distribution
  print('Conviction:')
  print(df_mypf['Conviction'].value_counts())

  # star stocks
  print('Star Stocks:')
  print(df_mypf['StarStock'].value_counts())

  # v40/v40n stocks
  print('V40/V40N Stocks:')
  print(df_mypf['Criteria'].value_counts())

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())

  # stocks in portfolio
  print('Stocks in Portfolio:')
  print(df_mypf['InPortfolio'].value_counts())

  # star stocks in portfolio
  print('Star Stocks in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['StarStock'].value_counts())

  # criteria of portfolio stocks
  print('Criteria in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Criteria'].value_counts())

  # strategies in portfolio stocks
  print('Strategy in Portfolio:')
  print(df_mypf[(df_mypf['InPortfolio'] == 1)]['Strategy'].value_counts())

print_distribution()

Conviction:
M    77
L    71
H    37
Name: Conviction, dtype: int64
Star Stocks:
0.0    94
1.0    89
Name: StarStock, dtype: int64
V40/V40N Stocks:
V40N    39
V40     38
2T3Y    10
SS       9
V200     6
TBD      3
ATH      1
Name: Criteria, dtype: int64
Latest Qtr:
0    185
Name: LatestQtr, dtype: int64
Star + V40/V40N Stocks:
1.0    41
0.0    36
Name: StarStock, dtype: int64
Stocks in Portfolio:
0    139
1     46
Name: InPortfolio, dtype: int64
Star Stocks in Portfolio:
1.0    32
0.0    14
Name: StarStock, dtype: int64
Criteria in Portfolio:
V40     11
V40N    10
2T3Y    10
SS       8
V200     3
TBD      3
ATH      1
Name: Criteria, dtype: int64
Strategy in Portfolio:
ATH     31
2T3Y    14
Name: Strategy, dtype: int64


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000

  return current_pe


In [9]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    #tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    #tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [10]:

# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-06-08 00:00:00+05:30,18635.0,^NSEI,4.29,0.29,0.85,19497.0,7610.0,7.25,5.0
2023-06-15 00:00:00+05:30,18688.0,^NSEI,4.45,0.26,0.88,19497.0,7610.0,6.81,4.0
2023-06-22 00:00:00+05:30,18771.0,^NSEI,4.73,0.31,0.67,19497.0,7610.0,6.11,4.0
2023-06-30 00:00:00+05:30,19189.0,^NSEI,6.88,0.35,0.62,19497.0,7610.0,2.59,2.0
2023-07-07 00:00:00+05:30,19497.0,^NSEI,8.37,0.37,0.92,19497.0,7610.0,0.00,0.0


In [11]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

df_mypf['Stock'] = df_mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [12]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [14]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 46


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1067.0,-27.51,-4.69,-2.55,94.94,304.0,1225.87,45.2,21.40,L,53.0,0,0.0,2T3Y,2T3Y
VALIANTORG.NS,547.0,-2.64,-1.42,0.92,88.65,238.0,542.52,24.0,17.50,L,27.0,0,0.0,2T3Y,2T3Y
BANDHANBNK.NS,231.0,-3.18,-1.26,0.41,86.47,213.0,253.98,NaN,1000.00,M,NaN,0,1.0,2T3Y,2T3Y
VAIBHAVGBL.NS,313.0,-1.65,-0.94,-0.32,76.02,209.0,317.48,35.4,49.30,L,-39.0,0,0.0,2T3Y,2T3Y
LUXIND.NS,1499.0,-1.90,-1.11,1.22,82.42,203.0,1493.72,21.4,32.90,M,-54.0,0,0.0,V40N,2T3Y
NIACL.NS,120.0,12.01,0.93,1.69,81.33,152.0,117.07,NaN,18.80,L,NaN,0,0.0,TBD,ATH
METROPOLIS.NS,1478.0,7.72,0.15,1.79,77.12,133.0,1365.98,NaN,52.80,L,NaN,0,0.0,TBD,ATH
TEAMLEASE.NS,2429.0,-2.61,-1.76,1.19,75.03,117.0,2279.93,NaN,36.60,M,NaN,0,1.0,V40N,2T3Y
INDIGOPNTS.NS,1459.0,12.73,-0.46,2.10,77.68,113.0,1407.73,104.9,52.70,M,50.0,0,1.0,V40N,2T3Y


In [15]:
# top v40/v40n stocks from portfolio for SIP
tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')][tmp_df['Dev%_200'] < 0].sort_values(by = 'Upside%ATH', ascending=False)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
LUXIND.NS,1499.0,-1.90,-1.11,1.22,82.42,203.0,1493.72,21.4,32.9,M,-54.0,0,0.0,V40N,2T3Y
TEAMLEASE.NS,2429.0,-2.61,-1.76,1.19,75.03,117.0,2279.93,NaN,36.6,M,NaN,0,1.0,V40N,2T3Y
RAJESHEXPO.NS,532.0,-20.25,-0.30,-0.53,81.93,78.0,624.78,NaN,11.0,M,NaN,0,1.0,V40N,ATH
FINEORG.NS,4842.0,-8.87,-1.66,1.16,36.43,49.0,4697.43,NaN,24.0,M,NaN,0,1.0,V40N,ATH
PAGEIND.NS,37283.0,-12.16,-1.38,-0.65,43.43,44.0,39325.03,97.7,72.8,H,25.0,0,1.0,V40,ATH
INFY.NS,1339.0,-4.72,-0.43,1.18,37.21,38.0,1263.00,29.0,23.1,H,20.0,0,1.0,V40,ATH


In [16]:
# top 10 from portfolio for SIP
tmp_df[tmp_df['Dev%_200'] < 0].sort_values(by = 'Upside%ATH', ascending=False).head(10)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1067.0,-27.51,-4.69,-2.55,94.94,304.0,1225.87,45.2,21.4,L,53.0,0,0.0,2T3Y,2T3Y
VALIANTORG.NS,547.0,-2.64,-1.42,0.92,88.65,238.0,542.52,24.0,17.5,L,27.0,0,0.0,2T3Y,2T3Y
BANDHANBNK.NS,231.0,-3.18,-1.26,0.41,86.47,213.0,253.98,NaN,1000.0,M,NaN,0,1.0,2T3Y,2T3Y
VAIBHAVGBL.NS,313.0,-1.65,-0.94,-0.32,76.02,209.0,317.48,35.4,49.3,L,-39.0,0,0.0,2T3Y,2T3Y
LUXIND.NS,1499.0,-1.90,-1.11,1.22,82.42,203.0,1493.72,21.4,32.9,M,-54.0,0,0.0,V40N,2T3Y
TEAMLEASE.NS,2429.0,-2.61,-1.76,1.19,75.03,117.0,2279.93,NaN,36.6,M,NaN,0,1.0,V40N,2T3Y
AAVAS.NS,1541.0,-13.52,-2.19,-0.56,64.62,107.0,1390.60,NaN,28.3,M,NaN,0,1.0,2T3Y,2T3Y
CLEAN.NS,1375.0,-7.18,-1.62,-0.07,90.35,93.0,1396.49,NaN,49.6,L,NaN,0,1.0,2T3Y,2T3Y
IRCTC.NS,634.0,-3.27,-0.46,0.47,51.66,84.0,633.05,77.4,51.4,M,34.0,0,1.0,SS,ATH


In [17]:
# top 5 from portfolio to book profit
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ASIANPAINT.NS,3402.0,12.33,-0.10,1.61,5.73,4.0,3091.34,87.9,79.1,H,10.0,0,1.0,V40,ATH
RELIANCE.NS,2664.0,8.21,-0.04,1.04,7.56,6.0,2467.15,29.1,27.0,H,7.0,0,1.0,V40,ATH
PIDILITIND.NS,2653.0,4.78,-0.47,0.74,12.43,9.0,2504.06,94.6,106.0,H,-12.0,0,1.0,V40,ATH
TCS.NS,3315.0,3.17,0.28,0.52,23.38,16.0,3231.00,28.1,28.8,H,-2.0,0,1.0,V40,ATH
BAJAJFINSV.NS,1616.0,8.46,-0.60,1.85,19.32,18.0,1454.93,39.2,40.1,H,-2.0,0,1.0,V40,ATH
ICICIGI.NS,1345.0,16.22,0.43,2.27,28.69,20.0,1137.80,46.2,1000.0,H,-2065.0,0,1.0,V40,ATH
BATAINDIA.NS,1719.0,7.24,-0.75,1.27,32.57,26.0,1523.62,126.0,68.4,H,46.0,0,0.0,V40,ATH


In [18]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)
tmp_base_df = df_prec_dev[~df_prec_dev.index.isin(df_mypf[df_mypf['InPortfolio'] == 1].Stock.values)]

In [19]:
# star + v40/v40n stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 26)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
DIXON.NS,4269.0,16.54,-0.08,3.72,5750.0,374.0,27.55,35.0,M,0,1.0,V40N,NaN
BERGEPAINT.NS,682.0,12.72,0.17,1.23,858.0,260.0,29.43,26.0,M,0,1.0,V40,NaN


In [20]:
# top 5 from star + v40/v40n for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
BERGEPAINT.NS,682.0,12.72,0.17,1.23,858.0,260.0,29.43,26.0,M,0,1.0,V40,NaN
DIXON.NS,4269.0,16.54,-0.08,3.72,5750.0,374.0,27.55,35.0,M,0,1.0,V40N,NaN


In [21]:
# star + v40/v40n stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
HDFCBANK.NS,1669.0,5.46,0.63,0.24,1728.0,747.0,6.01,4.0,H,0,1.0,V40,NaN
BAJFINANCE.NS,7714.0,17.95,0.11,2.57,7860.0,1804.0,2.41,2.0,H,0,1.0,V40,ATH
COLPAL.NS,1765.0,14.16,0.39,1.29,1794.0,931.0,3.36,2.0,H,0,1.0,V40,NaN
ABBOTINDIA.NS,23679.0,13.38,1.18,0.42,23826.0,6634.0,0.86,1.0,M,0,1.0,V40,NaN
AKZOINDIA.NS,2478.0,9.22,1.06,0.66,2508.0,1360.0,2.61,1.0,M,0,1.0,V40,NaN
BAJAJ-AUTO.NS,4888.0,30.73,1.28,1.64,4918.0,1695.0,0.93,1.0,L,0,1.0,V40,NaN
ICICIBANK.NS,953.0,6.28,0.22,0.53,959.0,252.0,0.85,1.0,H,0,1.0,V40,NaN
POLYCAB.NS,3535.0,20.40,1.67,1.20,3575.0,523.0,1.31,1.0,M,0,1.0,V40N,NaN
EQUITASBNK.NS,93.0,45.31,3.12,2.41,93.0,33.0,0.00,0.0,M,0,1.0,V40N,NaN


In [22]:
# star stocks (for upside)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
ADANITRANS.NS,770.0,-59.15,-8.44,-2.61,4105.0,148.0,84.28,433.0,L,0,1.0,NaN,NaN
ADANIGREEN.NS,959.0,-32.32,-4.87,0.32,2970.0,30.0,68.40,210.0,L,0,1.0,NaN,NaN
ADANIENT.NS,2404.0,-13.51,-2.05,2.52,4161.0,71.0,42.96,73.0,M,0,1.0,NaN,NaN
SBICARD.NS,847.0,4.43,-0.62,0.80,1133.0,496.0,44.90,34.0,M,0,1.0,NaN,NaN
GODREJIND.NS,508.0,14.36,0.23,1.26,644.0,235.0,33.25,27.0,L,0,1.0,NaN,NaN
UPL.NS,670.0,-6.69,-0.56,-0.73,832.0,245.0,27.60,24.0,L,0,1.0,NaN,NaN
TATAPOWER.NS,231.0,9.57,0.00,1.41,285.0,26.0,20.85,23.0,M,0,1.0,NaN,NaN


In [24]:
# star stocks (for breakout)

tmp_df = tmp_base_df[tmp_base_df.index.isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
EQUITASBNK.NS,93.0,45.31,3.12,2.41,93.0,33.0,0.00,0.0,M,0,1.0,V40N,NaN
HINDUNILVR.NS,2761.0,7.73,0.27,0.99,2761.0,1394.0,0.00,0.0,H,0,1.0,V40,NaN
ITC.NS,473.0,26.78,1.88,1.37,475.0,124.0,0.57,0.0,H,0,1.0,V40,NaN
TITAN.NS,3200.0,22.57,0.77,1.76,3200.0,730.0,0.00,0.0,H,0,1.0,V40,NaN
ABBOTINDIA.NS,23679.0,13.38,1.18,0.42,23826.0,6634.0,0.86,1.0,M,0,1.0,V40,NaN
AKZOINDIA.NS,2478.0,9.22,1.06,0.66,2508.0,1360.0,2.61,1.0,M,0,1.0,V40,NaN
BAJAJ-AUTO.NS,4888.0,30.73,1.28,1.64,4918.0,1695.0,0.93,1.0,L,0,1.0,V40,NaN
ICICIBANK.NS,953.0,6.28,0.22,0.53,959.0,252.0,0.85,1.0,H,0,1.0,V40,NaN
POLYCAB.NS,3535.0,20.40,1.67,1.20,3575.0,523.0,1.31,1.0,M,0,1.0,V40N,NaN


In [25]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
PGHH.NS,14800.0,6.55,0.08,0.49,15778.0,8145.0,12.81,7.0,L,0,0.0,V40,NaN
NESTLEIND.NS,22872.0,13.97,0.95,0.98,23099.0,8621.0,1.57,1.0,H,0,0.0,V40,NaN


In [26]:
# stocks prospects

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = tmp_base_df[tmp_base_df.index.isin(excluded_stocks)]
#tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,


In [27]:
# prospects with heavy upside
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 10


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1057.0,-8.42,-2.34,-0.93,3219.0,1017.0,98.18,205.0,M,0,0.0,V40N,NaN
WHIRLPOOL.NS,1475.0,2.15,-1.04,1.13,2667.0,1254.0,84.36,81.0,M,0,0.0,V40,NaN
LALPATHLAB.NS,2345.0,8.36,-0.46,1.85,4186.0,807.0,54.48,79.0,M,0,0.0,V40N,NaN
SFL.NS,1183.0,-2.39,-1.57,1.19,1972.0,566.0,56.12,67.0,M,0,0.0,V40N,NaN
SYMPHONY.NS,886.0,-5.87,-0.11,-1.11,1470.0,707.0,76.54,66.0,M,0,0.0,V40N,NaN
PFIZER.NS,3797.0,-6.01,-0.59,0.05,6009.0,2298.0,59.61,58.0,L,0,0.0,V40,NaN
TTKPRESTIG.NS,760.0,-5.01,-1.62,0.42,1165.0,391.0,52.33,53.0,M,0,0.0,V40N,NaN
MOTILALOFS.NS,745.0,12.58,-0.45,1.98,1091.0,439.0,53.07,46.0,L,0,0.0,V40N,NaN
NAM-INDIA.NS,288.0,22.95,0.00,2.54,413.0,111.0,41.39,43.0,L,0,0.0,V40,NaN


In [28]:
# prospects near 52W low
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 4


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1057.0,-8.42,-2.34,-0.93,3219.0,1017.0,98.18,205.0,M,0,0.0,V40N,NaN
WHIRLPOOL.NS,1475.0,2.15,-1.04,1.13,2667.0,1254.0,84.36,81.0,M,0,0.0,V40,NaN
SYMPHONY.NS,886.0,-5.87,-0.11,-1.11,1470.0,707.0,76.54,66.0,M,0,0.0,V40N,NaN
SIS.NS,424.0,9.23,-0.52,1.00,605.0,321.0,63.73,43.0,M,0,0.0,V40N,NaN


In [29]:
# prospects near 52W high
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 24


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
TITAN.NS,3200.0,22.57,0.77,1.76,3200.0,730.0,0.00,0.0,H,0,1.0,V40,NaN
AXISBANK.NS,986.0,11.72,1.13,1.18,986.0,302.0,0.00,0.0,M,0,0.0,V40,NaN
HINDUNILVR.NS,2761.0,7.73,0.27,0.99,2761.0,1394.0,0.00,0.0,H,0,1.0,V40,NaN
EQUITASBNK.NS,93.0,45.31,3.12,2.41,93.0,33.0,0.00,0.0,M,0,1.0,V40N,NaN
ITC.NS,473.0,26.78,1.88,1.37,475.0,124.0,0.57,0.0,H,0,1.0,V40,NaN
ICICIBANK.NS,953.0,6.28,0.22,0.53,959.0,252.0,0.85,1.0,H,0,1.0,V40,NaN
ABBOTINDIA.NS,23679.0,13.38,1.18,0.42,23826.0,6634.0,0.86,1.0,M,0,1.0,V40,NaN
BAJAJ-AUTO.NS,4888.0,30.73,1.28,1.64,4918.0,1695.0,0.93,1.0,L,0,1.0,V40,NaN
POLYCAB.NS,3535.0,20.40,1.67,1.20,3575.0,523.0,1.31,1.0,M,0,1.0,V40N,NaN


In [30]:
# prospects in bull run
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 24


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
ANGELONE.NS,1775.0,31.87,0.97,3.85,1889.0,207.0,6.78,6.0,M,0,1.0,V40N,NaN
ISEC.NS,608.0,21.84,1.00,3.28,829.0,164.0,33.23,36.0,M,0,0.0,V40N,NaN
BAJFINANCE.NS,7714.0,17.95,0.11,2.57,7860.0,1804.0,2.41,2.0,H,0,1.0,V40,ATH
HDFCLIFE.NS,665.0,20.89,0.73,2.21,754.0,339.0,21.45,13.0,H,0,0.0,V40,NaN
SANOFI.NS,6942.0,22.63,1.04,2.03,8070.0,4018.0,27.84,16.0,M,0,0.0,V40,NaN
OFSS.NS,3862.0,25.02,1.46,1.93,4392.0,1232.0,16.77,14.0,M,0,1.0,V40N,NaN
CERA.NS,7548.0,23.89,1.76,1.92,8036.0,2065.0,8.17,6.0,H,0,1.0,V40N,NaN
MCDOWELL-N.NS,936.0,11.98,0.48,1.87,1004.0,448.0,12.23,7.0,L,0,1.0,V40N,NaN
3MINDIA.NS,27220.0,16.17,1.09,1.87,29373.0,15332.0,15.33,8.0,M,0,1.0,V40N,NaN


In [31]:
# prospects about to turn around
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 15


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
DIXON.NS,4269.0,16.54,-0.08,3.72,5750.0,374.0,27.55,35.0,M,0,1.0,V40N,NaN
KANSAINER.NS,315.0,11.00,-0.70,2.11,436.0,192.0,49.59,38.0,M,0,0.0,V40N,NaN
MOTILALOFS.NS,745.0,12.58,-0.45,1.98,1091.0,439.0,53.07,46.0,L,0,0.0,V40N,NaN
LALPATHLAB.NS,2345.0,8.36,-0.46,1.85,4186.0,807.0,54.48,79.0,M,0,0.0,V40N,NaN
TATAELXSI.NS,7586.0,10.18,-0.92,1.82,10602.0,525.0,29.93,40.0,M,0,0.0,V40N,NaN
SFL.NS,1183.0,-2.39,-1.57,1.19,1972.0,566.0,56.12,67.0,M,0,0.0,V40N,NaN
WHIRLPOOL.NS,1475.0,2.15,-1.04,1.13,2667.0,1254.0,84.36,81.0,M,0,0.0,V40,NaN
SIS.NS,424.0,9.23,-0.52,1.00,605.0,321.0,63.73,43.0,M,0,0.0,V40N,NaN
HAVELLS.NS,1301.0,6.06,-0.24,0.69,1456.0,451.0,15.42,12.0,H,0,0.0,V40,NaN


In [32]:
# cross-check the features
tmp_df = tmp_base_df[(tmp_base_df['Criteria'] == 'V40') | (tmp_base_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 10


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
JCHAC.NS,1057.0,-8.42,-2.34,-0.93,3219.0,1017.0,98.18,205.0,M,0,0.0,V40N,NaN
WHIRLPOOL.NS,1475.0,2.15,-1.04,1.13,2667.0,1254.0,84.36,81.0,M,0,0.0,V40,NaN
LALPATHLAB.NS,2345.0,8.36,-0.46,1.85,4186.0,807.0,54.48,79.0,M,0,0.0,V40N,NaN
SFL.NS,1183.0,-2.39,-1.57,1.19,1972.0,566.0,56.12,67.0,M,0,0.0,V40N,NaN
SYMPHONY.NS,886.0,-5.87,-0.11,-1.11,1470.0,707.0,76.54,66.0,M,0,0.0,V40N,NaN
PFIZER.NS,3797.0,-6.01,-0.59,0.05,6009.0,2298.0,59.61,58.0,L,0,0.0,V40,NaN
TTKPRESTIG.NS,760.0,-5.01,-1.62,0.42,1165.0,391.0,52.33,53.0,M,0,0.0,V40N,NaN
MOTILALOFS.NS,745.0,12.58,-0.45,1.98,1091.0,439.0,53.07,46.0,L,0,0.0,V40N,NaN
NAM-INDIA.NS,288.0,22.95,0.00,2.54,413.0,111.0,41.39,43.0,L,0,0.0,V40,NaN


In [33]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,
PVRINOX.NS,1407.0,-12.18,-0.87,-1.12,1917.0,1348.0,89.63,36.0,L,0,0.0,NaN,NaN


In [34]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [35]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-06-08 00:00:00+05:30,836.0,BHARTIARTL.NS,5.96,0.76,1.01,879.0,258.0,6.92,5.0
2023-06-15 00:00:00+05:30,830.0,BHARTIARTL.NS,4.76,0.76,1.00,879.0,258.0,7.89,6.0
2023-06-22 00:00:00+05:30,843.0,BHARTIARTL.NS,6.15,0.63,0.87,879.0,258.0,5.80,4.0
2023-06-30 00:00:00+05:30,879.0,BHARTIARTL.NS,10.26,0.63,1.11,879.0,258.0,0.00,0.0
2023-07-07 00:00:00+05:30,870.0,BHARTIARTL.NS,8.84,0.63,1.33,879.0,258.0,1.45,1.0
